<a href="https://colab.research.google.com/github/momo54/minfo2022/blob/main/MasterInfo_UE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# grrr

Bon...
L'extraction excel de pdf ne peut être miraculeux..

Quand un tableau est à cheval sur plusieurs page, le tableau est mal reconstitué...

Il faut avoir les tableaux d'origine !!

Donc un read-Excel ne pourra donner de bons résultats...

In [ ]:
!pip install pyspark
!pip install -q findspark
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 38 kB/s 
     |████████████████████████████████| 199 kB 40.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=6c3f239f2e6b86cd20785fd1226a959a6c0c137c0c43b7455de68811b1e57c81
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


# Building Table UES_T

UE with description, parcours, année

In [ ]:
import pandas as pd

##
## Autre Approche
## Même si 1 sheet != 1 UE
## l'odre est préservé
## regexp extraction en suivant l'ordre...

my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

df=pd.read_excel(my_csv, index_col=None, header=None, sheet_name="Table 5")
#Title and code of UE not at the right Place :(
# need swapping...
#print(df)
#print(df.dtypes)
mask=df.apply(lambda x: x.str.contains('X1.*|Objectifs|Contenu',regex=True)).any(axis=1)
print(df[mask])



                                        0  \
1                                 X1II010   
14  Objectifs (résultats d'apprentissage)   
15                                Contenu   
19                                X1II030   
32  Objectifs (résultats d'apprentissage)   
33                                Contenu   

                                                    1  
1                                             Graphes  
14  1. reconnaître et modéliser un problème réel a...  
15  •\nEléments de théorie des graphes\n•\nArbres ...  
19                          Complexité et algorithmes  
32  • Savoir manipuler les notations standards des...  
33                                                NaN  


In [ ]:
#
# DS M2 et Vico M2 n'ont pas les code UE pour les UE...
# added it by hand in excel files.... -> withcode.xlsx
#

parcours=[
    ('ALMA','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'),
    ('ALMA','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx'),
    ('ATAL','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx'),
    ('ATAL','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx'),
    ('ORO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('ORO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx'),
    ('DS','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Data_Science_(DS)_2022-2023.xlsx'),
    ('DS','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Data_Science_(DS)_2022-2023-withcode.xlsx'),
    ('VICO','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_Visual_Computing_(VICO)_2022-2023.xlsx'),
    ('VICO','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_Visual_Computing_(VICO)-withcode.xlsx'),
    ('CMI','M1','/content/drive/MyDrive/MasterInfo2022/converted/M1_CMI-OPTIM_2022-2023.xlsx'),
    ('CMI','M2','/content/drive/MyDrive/MasterInfo2022/converted/M2_CMI-OPTIM_2022-2023.xlsx')
]


def extract_ue(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  for sheet_name, df in pd.read_excel(path, index_col=None, header=None, sheet_name=None).items():
      sheetid=int(sheet_name.split(" ")[1])
      if sheetid>4:
        print(sheet_name)
        dfs.append(df)
  allUE=pd.concat(dfs).reset_index(drop=True)

  mask=allUE.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*|Object|Contenu',regex=True)).any(axis=1)
  UES=allUE[mask]

  #debug
  #print(UES.columns.values)
  #print(UES)
  #i don't know why i need 3 colmns sometimes...
  # read_excel pb ???
  if len(UES.columns.values)==2:
    UES.set_axis(['X','Y'],axis=1,inplace=True)
  else:
    UES.set_axis(['X','Y','Z'],axis=1,inplace=True)
    
  #debug
  #print(UES.columns.values)
  #print(UES)
  #for index, row in UES.iterrows():
  #  print(f"extracting:{row['X']}")

  state=0
  key=""
  title=""
  objective=""
  content=""
  lines=[]

  import re
  reg=re.compile("^X[1-4].*|^ZZ|^YG.*")

  for index, row in UES.iterrows():
    #print(f"extracting:{row['X']}")
    #print(row['X'],row['Y']
    if state==0:
      if reg.match(str(row['X'])) is not None:
        key=row['X']
        title=row['Y']
        state=1
      else:
        print(f"error: should read an UE key:{row['X']}")
        state=0
    elif state==1:
      if str(row['X']).startswith("Object"):
        objective=row['Y']
        state=2
      else:
        print(f"error: should read an Objective:{row['X']}")
        state=0
    elif state==2:
      if str(row['X']).startswith("Contenu"):
        content=row['Y']
        lines.append({'key' : key, 'title':title,'objective':objective,'content':content,'parc':parcours,'an':an,'from':path})
        state=0
      else:
        print(f"error: should read an Content:{row['X']}")
        state=0
        
  out=pd.DataFrame(lines)
  out.to_csv(f"{parcours}_{an}_UEs.csv",sep='|',index=False,header=True)
  return out

dfs=[]
for p,a,path in parcours:
  df=extract_ue(p,a,path)
  dfs.append(df)

all=pd.concat(dfs)
print(all)


extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
Table 5
Table 6
Table 7
Table 8
Table 9
Table 10
Table 11
Table 12
Table 13
Table 14
Table 15
Table 16
Table 17
Table 18
Table 19
Table 20
Table 21
Table 22
Table 23
Table 24
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(A

In [ ]:
# OK CSV shoud be generated now
# Read UE in SPARK to groupby UE CODE...
# Troubles when reading CSV in spark... Some end of fields not well detected...
# Walk around on next block of code... (NO CSV)

from pyspark.sql.types import *
schema = StructType([
    StructField("key",StringType(),True), 
    StructField("title", StringType(), True),
    StructField("objective", StringType(), True),
    StructField("content", StringType(), True),
    StructField("parc", StringType(), True),
    StructField("an", StringType(), True),
    StructField("path", StringType(), True),
    ])

file_list=[]
for pa,an,path in parcours:
  file_list.append(f"/content/{pa}_{an}_UEs.csv")

#print(','.join(file_list))

data = spark.read \
    .format("csv")   \
    .option("delimiter","|") \
    .option("multiLine", True)\
    .option("header",True) \
    .schema(schema) \
    .load(file_list)
print(data.count())
data.show(209)

data.createOrReplaceTempView(f"UES")

UES_table=spark.sql("select key,title,objective, collect_set(parc), collect_set(an) from UES  group by key,title,objective,content")

UES_table.createOrReplaceTempView(f"UES_T")
#don't understand why pb in CSV...
UES_table.show(5)





252
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|                 key|               title|           objective|             content|                parc|  an|                path|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+--------------------+
|             X1II010|             Graphes|1. reconnaître et...|•\nEléments de th...|                 CMI|  M1|/content/drive/My...|
|             X1II030|Complexité et alg...|• Savoir manipule...|                null|                 CMI|  M1|/content/drive/My...|
|             X1II040|Anglais scientifique|Au terme de cette...|Anglais de spécia...|                 CMI|  M1|/content/drive/My...|
|             X1II050|Communication, co...|                null|                null|                 CMI|  M1|/content/drive/My...|
|             X1II060|Introduction à la...|                null| 

In [ ]:
## Work on UE data with SPARK now...

mydata=spark.createDataFrame(all.astype(str))
#mydata.show(5)

mydata.createOrReplaceTempView(f"UES")

ues=spark.sql("select key,title,objective, content, concat_ws(',',collect_set(parc)) as parcours, concat_ws(',',collect_set(an)) as an from UES  group by key,title,objective,content")
ues.show(5) 

ues.createOrReplaceTempView(f"UES_T")

!rm -rf ues-uniq.csv

ues.toPandas().to_csv('ues-uniq.csv')
#ues.write.options(header='True',delimiter='|') \
#  .csv("ues-uniq.csv")


+-------+--------------------+--------------------+--------------------+--------------------+---+
|    key|               title|           objective|             content|            parcours| an|
+-------+--------------------+--------------------+--------------------+--------------------+---+
|X1I2020|Graphes II et Rés...|1. reconnaître, m...|•\nProblème du fl...|ATAL,ORO,DS,CMI,VICO| M1|
|X1II040|Anglais scientifique|Au terme de cette...|Anglais de spécia...|ATAL,ORO,DS,CMI,A...| M1|
|X2II020|         Data Mining|A l'issue de ce c...|Le Data Mining es...|ATAL,ORO,DS,CMI,A...| M1|
|X1I1020|       Test logiciel|• To be able to c...|• Functional and ...|                ALMA| M1|
|X1LI010|Management à Visé...|A l’issue de cet ...|Autour d’une form...|   ATAL,DS,ALMA,VICO| M1|
+-------+--------------------+--------------------+--------------------+--------------------+---+
only showing top 5 rows



# Building ECTS_T

UE -> ECTS, Hours

In [ ]:
# ok now lets extracts ECTS and hours linked to UE from excel files

def extract_ects(parcours,an,path):
  print(f"extracting ues for {parcours}, annee:{an}, path:{path}")
  dfs=[]
#  my_csv=r'/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx'

  data=pd.read_excel(path, index_col=None, header=1, sheet_name="Table 2")
  mask=data.apply(lambda x: x.str.contains('^X[1-4].*|^ZZ|^YG.*',regex=True)).any(axis=1)
  ects=data[mask]
  #print(ects)
  return ects

dfs=[]
for p,a,path in parcours:
  df=extract_ects(p,a,path)
  dfs.append(df)

ects=pd.concat(dfs)
print(ects)



extracting ues for ALMA, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Architecture_Logicielle_(ALMA)_2022-2023.xlsx
extracting ues for ALMA, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Architectures_Logicielles_(ALMA)_2022-2023.xlsx
extracting ues for ATAL, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023 (2).xlsx
extracting ues for ATAL, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Apprentissage_et_Traitement_Automatique_de_la_Langue_(ATAL)_2022-2023.xlsx
extracting ues for ORO, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converted/M1_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for ORO, annee:M2, path:/content/drive/MyDrive/MasterInfo2022/converted/M2_Optimisation_en_Recherche_Operationnelle_(ORO)_2022-2023.xlsx
extracting ues for DS, annee:M1, path:/content/drive/MyDrive/MasterInfo2022/converte

In [ ]:
## Work with ECTS with SPARK

#from pyspark.sql.types import *
#ects_schema = StructType([
#    StructField("0", StringType(), True)\
#    ,StructField("1", StringType(), True)
#    ,StructField("2", LongType(), True)
#    ,StructField("3", StringType(), True)
#    ,StructField("4", StringType(), True)
#    ,StructField("5", StringType(), True)
#    ,StructField("6", StringType(), True)
#    ,StructField("8", LongType(), True)
#    ,StructField("9", StringType(), True)
#    ,StructField("10", StringType(), True)
#    ])

s_ects=spark.createDataFrame(ects.astype(str))
#s_ects=spark.createDataFrame(ects,schema=ects_schema)

print(f"Reading {s_ects.count()} in ECTS tablles")

s_ects=s_ects.withColumnRenamed("0","title") \
  .withColumnRenamed("Code","key") \
  .withColumnRenamed("ECTS","ects") \
  .withColumnRenamed("Total","hours") \



s_ects.createOrReplaceTempView(f"ECTS")

ects_table=spark.sql("select key,  cast(max(ects) as float) as ects, cast(max(hours) as float) as hours from ECTS  group by key,ects,hours")
ects_table.createOrReplaceTempView(f"ECTS_T")
ects_table.show(5)

print(f"produced {ects_table.count()} in ECTS_T")


ects_table.toPandas().to_csv('ects.csv')


Reading 240 in ECTS tablles
+-------+----+-----+
|    key|ects|hours|
+-------+----+-----+
|X1CI010| 2.0| 22.0|
|X1I1010| 6.0| 60.0|
|X1I1020| 3.0| 24.0|
|X1I1030| 4.0| 32.0|
|X1I1030| 5.0| 32.0|
+-------+----+-----+
only showing top 5 rows

produced 104 in ECTS_T


# BUILDING ACM_T

In [31]:
# I suppose now that UE has be aligned with ACM classif...
# Read the excel file and group yby ACM items

path='/content/drive/MyDrive/MasterInfo2022/ues-uniq.xlsx'
data=pd.read_excel(path, index_col=None, header=1)

#swtich ain spark now
s_acm=spark.createDataFrame(data.astype(str))
s_acm.show(5)

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum

#s_acm=s_acm.select("key",concat_ws("|",*[when(col(c)=="x",c).alias(c) for c in s_acm.columns]).alias("types"))
# hmmm, bricolage...
s_acm=s_acm.select("key", "title", array_remove(array([when(col(c)=="x",c).otherwise('#').alias(c) for c in s_acm.columns]),'#').alias("types"))

s_acm.show(5,truncate=200)

s_acm.createOrReplaceTempView("ACM_ARRAY")
s_acm=spark.sql("select key, title ,types, size(types) as nbtypes from ACM_ARRAY")

s_acm.show(5,truncate=200)

s_acm=s_acm.select("key",explode("types").alias("types"),"nbtypes")
s_acm.show(100,truncate=200)

s_acm.createOrReplaceTempView("ACM_T")


+----------+-------+--------------------+--------------------+--------------------+--------------------+---+--------------------------+------------------------------+----------+-----------------------------+---------------------------------+-------------------------------------------------+-----------+----------------------+----------------------+------------------+--------+---------------------------------+-------------+-------------------+-----------------+------------------------+-------------------------------+-------------------------------+------------------------------+-------------------------+----------------------------+------------------------+-------------------------+---------------------------------------+-----------------+-------+------------+
|Unnamed: 0|    key|               title|           objective|             content|            parcours| an|T_MSF (Proof, graph proba)|T_AL: Algorithm and Complexity|T_Modeling|T_PL: prog and formal methods|CS_Computer architecture

# Prepare data for Plotting

In [40]:
tmptypes=spark.sql("select types, 0 as ects, 0 as hours from ACM_T group by types")
tmptypes.createOrReplaceTempView("TMPTYPES")
print(f"number of types:{tmptypes.count()}")

tmpprefix=spark.sql("select substring(ACM_T.types,1,2) as prefix, 0 as ects, 0 as hours from ACM_T group by substring(ACM_T.types,1,2)")
tmpprefix.createOrReplaceTempView("TMPPREFIX")
print(f"number of prefix:{tmpprefix.count()}")



number of types:26
number of prefix:5


In [44]:
# Bon plotting now ??

# carefull, we need to have all categories for each parcours !!
# create a temp table

# 1 M1 -> 450h et 60ects
# 1 M2 -> 261h et 60ects
# 1 M1 + M2 ->  711h et 120 ects
# j'ai compté qu'un étudiant faisait toutes les options -> il faut normaliser aux attendus pour que les parcours soient comp.

from pyspark.sql.functions import col, when,concat_ws, explode,split,array,array_remove,lit,sum


results_pa={}
for pa,an,path in parcours:
  print(f"parcours:{pa}, année:{an}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by ACM_T.types
  order by ACM_T.types
  """
  
  results_pa[pa]={}
  results_pa[pa+an]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_pa[pa+an].show(26)
  print(results_pa[pa+an].count())
#  results_pa[pa][an].select(sum("ects"),sum("hours")).show()

  nbects=results_pa[pa+an].select(sum('ects')).collect()[0][0]
  nbhours=results_pa[pa+an].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_pa[pa+an]=results_pa[pa+an].withColumn("ects",col("ects")*60/nbects)
  if an=='M1':
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*450/nbhours)
  else:
    results_pa[pa+an]=results_pa[pa+an].withColumn("hours",col("hours")*261/nbhours)


  results_pa[pa+an].select(sum("ects"),sum("hours")).show()



parc=set([p[0] for p in parcours])

results_p={}
for pa in parc:
  print(f"parcours:{pa}")
  query=f"""
  select ACM_T.types, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
  group by ACM_T.types
  order by ACM_T.types
  """
 
  results_p[pa]=spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"])
  results_p[pa].show(26)
  print(results_p[pa].count())

  nbects=results_p[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_p[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_p[pa]=results_p[pa].withColumn("ects",col("ects")*120/nbects)
  results_p[pa]=results_p[pa].withColumn("hours",col("hours")*711/nbhours)

  results_p[pa].select(sum("ects"),sum("hours")).show()




parcours:ALMA, année:M1
+--------------------+----+------------------+
|               types|ects|             hours|
+--------------------+----+------------------+
|ACS_Commputer gra...| 0.0|               0.0|
|ACS_Computational...| 1.5|              12.0|
|ACS_Image and sou...| 0.0|               0.0|
|ACS_Natural Langu...| 0.0|               0.0|
|ACS_Social comput...| 4.0|30.857142857142858|
|ACS_Software engi...| 5.5| 48.85714285714286|
|ACS_Specialized P...| 7.0| 50.85714285714286|
|       AI_Classifier| 0.0|               0.0|
|            AI_Logic| 4.5|              32.0|
| AI_Machine learning| 2.0|23.994999885559082|
|   AI_Neural network| 1.0|11.994999885559082|
|AI_Proba for unce...| 0.0|               0.0|
|  AI_Search and opti| 1.5|              12.0|
|CS_Computer archi...| 3.5|26.857142857142858|
|CS_Concurrent, pa...| 3.5|26.857142857142858|
|CS_Database and m...|10.0| 74.85714285714286|
|         CS_Networks| 3.0|              24.0|
|CS_security and c...| 1.0| 6.857142

In [43]:
results_prefix={}
for pa in parc:
  print(f"prefix parcours:{pa}")

  query=f"""
    select substring(ACM_T.types,1,2) as prefix, 
        sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
        sum(ECTS_T.hours/ACM_T.nbtypes) as hours
    from ACM_T,ECTS_T, UES_T 
    where 
      ACM_T.key=ECTS_T.key 
      and ACM_T.key=UES_T.key 
      and UES_T.parcours like '%{pa}%'
    group by substring(ACM_T.types,1,2) 
    """
  results_prefix[pa]=spark.sql(query).union(tmpprefix).dropDuplicates(["prefix"]).orderBy(["prefix"])
  results_prefix[pa].show(26)
  print(results_prefix[pa].count())

  nbects=results_prefix[pa].select(sum('ects')).collect()[0][0]
  nbhours=results_prefix[pa].select(sum('hours')).collect()[0][0]
  print(f"ects:{nbects}")
  print(f"hours:{nbhours}")

  results_prefix[pa]=results_prefix[pa].withColumn("ects",col("ects")*120/nbects)
  results_prefix[pa]=results_prefix[pa].withColumn("hours",col("hours")*711/nbhours)

  results_prefix[pa].select(sum("ects"),sum("hours")).show()



prefix parcours:ALMA
+------+----+------------------+
|prefix|ects|             hours|
+------+----+------------------+
|    AC|29.5|231.57142857142858|
|    AI| 9.0| 79.98999977111816|
|    CS|32.0|247.42857142857144|
|    SO|60.0|             168.0|
|    T_|21.5|             210.0|
+------+----+------------------+

5
ects:152.0
hours:936.9899997711182
+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|120.00000000000001|     711.0|
+------------------+----------+

prefix parcours:DS
+------+----+------------------+
|prefix|ects|             hours|
+------+----+------------------+
|    AC|17.4| 134.9714285714286|
|    AI|25.9|214.38999977111817|
|    CS|31.2|236.12857142857143|
|    SO|32.0|             209.5|
|    T_|22.5|             204.0|
+------+----+------------------+

5
ects:129.0
hours:998.9899997711182
+---------+----------+
|sum(ects)|sum(hours)|
+---------+----------+
|    120.0|     711.0|
+---------+----------+

prefix parcou

# Plotting

In [ ]:
# Radar plot Now

import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories=flatten(results_p['ALMA'].select("types").collect())

pdata=[]
for p in parc:
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

pdata=[]
for p in parc:
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")



# plotting prefix

In [58]:
import plotly.graph_objects as go
import plotly.offline as pyo
import numpy as np

def flatten(l):
    return [str(item) for sublist in l for item in sublist]

#categories=np.array(results_p['ALMA'].select("types").orderBy("types").collect())
categories=flatten(results_prefix['ALMA'].select("prefix").collect())

pdata=[]
for p in parc:
  h=results_prefix[p].select("hours").rdd.map(lambda x: x.hours).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p,mode="lines+markers+text")
  pdata.append(entry)

fig_h = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info Hours'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_h.update_traces(fill='toself')
fig_h.show(renderer="colab")

pdata=[]
for p in parc:
  h=results_prefix[p].select("ects").rdd.map(lambda x: x.ects).collect()
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

fig_e = go.Figure(
      pdata,
      layout=go.Layout(
          title=go.layout.Title(text='Master Info ECTS'),
          polar={'radialaxis': {'visible': True}},
          showlegend=True
      )
  )

#pyo.plot(fig)
fig_e.update_traces(fill='toself')
fig_e.show(renderer="colab")



# plotting per Parcours


In [ ]:
# https://plotly.com/python/polar-chart/
for p in parc:

  results_p[p].select(sum("ects"),sum("hours")).show()

  pdata=[]
  h=results_p[p].select("hours").rdd.map(lambda x: x.hours).collect()

  print(f"hours:{h}")

  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Master Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 160]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  pdata=[]
  h=results_p[p].select("ects").rdd.map(lambda x: x.ects).collect()
  print(f"ects:{h}")
  entry=go.Scatterpolar(r=h, theta=categories, name=p)
  pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Master Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 35]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")


+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|119.99999999999999|     711.0|
+------------------+----------+

hours:[0.0, 9.10575353214457, 0.0, 0.0, 23.41479479694318, 95.50201025975437, 47.696804215995364, 0.0, 24.282009419052187, 18.20771291347818, 9.10195938133361, 0.0, 9.10575353214457, 53.767306570758414, 44.661553038613846, 65.90831128028451, 18.21150706428914, 5.20328773265404, 40.97589089465056, 65.25790031370275, 12.141004709526094, 9.10575353214457, 36.42301412857828, 27.31726059643371, 59.187397958939705, 36.42301412857828]


ects:[0.0, 1.1842105263157894, 0.0, 0.0, 3.1578947368421053, 12.236842105263158, 6.7105263157894735, 0.0, 3.5526315789473686, 1.5789473684210527, 0.7894736842105263, 0.0, 1.1842105263157894, 7.105263157894737, 5.921052631578948, 9.078947368421053, 2.3684210526315788, 0.7894736842105263, 4.7368421052631575, 19.736842105263158, 21.31578947368421, 1.5789473684210527, 3.5526315789473686, 2.763157894736842, 6.7105263157894735, 3.9473684210526314]


+------------------+----------+
|         sum(ects)|sum(hours)|
+------------------+----------+
|120.00000000000001|     711.0|
+------------------+----------+

hours:[0.0, 8.540626034249387, 2.2775002757998366, 5.69375068949959, 36.19598652610454, 4.880357733856792, 38.473486801904386, 44.98063044704677, 22.77500275799836, 44.97707177141608, 14.230818048118282, 17.081252068498774, 8.540626034249387, 19.11473445760577, 10.574108423356384, 116.40669936443004, 17.081252068498774, 4.880357733856792, 55.51406922262101, 44.12656784362183, 23.842581012279535, 25.62187810274816, 59.7843822397457, 42.70313017124693, 8.540626034249387, 34.16250413699755]


ects:[0.0, 1.3953488372093024, 0.18604651162790697, 0.9302325581395349, 6.046511627906977, 0.9302325581395349, 6.6976744186046515, 7.627906976744185, 4.186046511627907, 6.232558139534884, 1.8604651162790697, 2.7906976744186047, 1.3953488372093024, 3.255813953488372, 1.8604651162790697, 20.186046511627907, 2.7906976744186047, 0.9302325581395349, 3.7209302325581395, 8.372093023255815, 13.023255813953488, 4.651162790697675, 8.372093023255815, 6.976744186046512, 1.3953488372093024, 4.186046511627907]


+------------------+-----------------+
|         sum(ects)|       sum(hours)|
+------------------+-----------------+
|119.99999999999999|710.9999999999999|
+------------------+-----------------+

hours:[32.166184419610225, 8.041546104902556, 66.47678113386114, 24.12463831470767, 20.678261412606577, 12.63671530770402, 44.26679665365408, 23.588535241047502, 21.444122946406818, 50.390338203155494, 8.038195384002021, 5.3610307366017045, 8.041546104902556, 17.997746044305725, 9.956199939403167, 81.794011809866, 16.083092209805113, 4.595169202801461, 20.10386526225639, 63.327175576107635, 16.41815663084272, 8.041546104902556, 64.33236883922045, 32.166184419610225, 13.402576841504262, 37.52721515621193]


...

ects:[8.571428571428571, 1.0714285714285714, 11.571428571428571, 0.0, 2.857142857142857, 0.7142857142857143, 5.142857142857143, 3.357142857142857, 3.2142857142857144, 5.857142857142857, 0.7142857142857143, 0.0, 1.0714285714285714, 2.5, 1.4285714285714286, 10.5, 2.142857142857143, 1.4285714285714286, 2.857142857142857, 17.142857142857142, 19.285714285714285, 1.4285714285714286, 6.428571428571429, 6.428571428571429, 1.0714285714285714, 3.2142857142857144]


+------------------+-----------------+
|         sum(ects)|       sum(hours)|
+------------------+-----------------+
|119.99999999999999|711.0000000000001|
+------------------+-----------------+

hours:[0.0, 31.955355394071105, 0.0, 0.0, 15.977677697035553, 0.0, 13.314731414196293, 21.30357026271407, 3.994419424258888, 15.974348937994499, 7.9855100894767235, 0.0, 154.450884404677, 13.314731414196293, 5.325892565678518, 29.292409111231848, 15.977677697035553, 0.0, 19.972097121294443, 57.25334508104407, 10.651785131357036, 7.988838848517776, 174.42298152597147, 31.955355394071105, 11.983258272776665, 67.9051302124011]


ects:[0.0, 4.203821656050955, 0.0, 0.0, 2.2929936305732483, 0.0, 1.910828025477707, 3.4394904458598727, 0.5732484076433121, 1.5286624203821657, 0.7643312101910829, 0.0, 17.388535031847134, 1.910828025477707, 0.7643312101910829, 4.203821656050955, 2.2929936305732483, 0.0, 3.0573248407643314, 19.10828025477707, 20.636942675159236, 1.5286624203821657, 18.726114649681527, 4.585987261146497, 1.7197452229299364, 9.363057324840764]


+---------+----------+
|sum(ects)|sum(hours)|
+---------+----------+
|    120.0|     711.0|
+---------+----------+

hours:[0.0, 29.625257169576717, 0.0, 0.0, 14.812628584788358, 0.0, 12.3438571539903, 19.750171446384478, 3.7031571461970896, 14.809542549867745, 7.403228257473565, 0.0, 143.18874298628748, 12.3438571539903, 4.937542861596119, 27.15648573877866, 14.812628584788358, 0.0, 21.601750019483024, 82.08665007403549, 9.875085723192239, 7.406314292394179, 161.70452871727292, 49.3754286159612, 11.10947143859127, 62.953671485350526]


ects:[0.0, 3.815028901734104, 0.0, 0.0, 2.0809248554913293, 0.0, 1.7341040462427746, 3.121387283236994, 0.5202312138728323, 1.3872832369942196, 0.6936416184971098, 0.0, 15.780346820809248, 1.7341040462427746, 0.6936416184971098, 3.815028901734104, 2.0809248554913293, 0.0, 4.855491329479769, 20.809248554913296, 18.728323699421967, 4.161849710982659, 16.99421965317919, 6.936416184971098, 1.560693641618497, 8.497109826589595]


In [27]:
# https://plotly.com/python/polar-chart/
for p in parc:

  pdata=[]
  for an in ['M1','M2']:
    h=results_pa[p+an].select("hours").rdd.map(lambda x: x.hours).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
        pdata,
        layout=go.Layout(
            title=go.layout.Title(text=f"Master Info {p} Hours"),
            polar={'radialaxis': {'visible': True }},
            showlegend=True
        )
    )

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 160]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")

  pdata=[]
  for an in ['M1','M2']:
    h=results_pa[p+an].select("ects").rdd.map(lambda x: x.ects).collect()
    entry=go.Scatterpolar(r=h, theta=categories, name=f"{p} {an}")
    pdata.append(entry)

  fig = go.Figure(
    pdata,
    layout=go.Layout(
        title=go.layout.Title(text=f"Master Info {p} ECTS"),
        polar={'radialaxis': {'visible': True}},
        showlegend=True
    )
  )  

  #pyo.plot(fig)
  fig.update_polars(radialaxis=dict(range=[0, 35]))
  fig.update_traces(fill='toself')
  fig.show(renderer="colab")


In [38]:
#Debug, Debug...

#spark.sql("select * from UES_T where UES_T.parcours like '%DS%' and UES_T.an = 'M2'").show()
#spark.sql("select * from ACM_T where key like '%ZZ%'").show() 
#spark.sql("select * from ECTS_T where key like '%ZZ%'").show() 

## CMD + Crt + '+' to comment/uncomment
## Control ATAL
## in fact OK
# pa='ATAL'
# an='M2'
# query=f"""
#   select ACM_T.types, 
#       sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
#       sum(ECTS_T.hours/ACM_T.nbtypes) as hours
#   from ACM_T,ECTS_T, UES_T 
#   where 
#     ACM_T.key=ECTS_T.key 
#     and ACM_T.key=UES_T.key 
#     and UES_T.parcours like '%{pa}%'
#     and UES_T.an = '{an}'
#   group by ACM_T.types
#   order by ACM_T.types
#   """
  
# spark.sql(query).union(tmptypes).dropDuplicates(["types"]).orderBy(["types"]).show()

pa='ATAL'
an='M2'
query=f"""
  select substring(ACM_T.types,1,2) as prefix, 
      sum(ECTS_T.ects/ACM_T.nbtypes) as ects,
      sum(ECTS_T.hours/ACM_T.nbtypes) as hours
  from ACM_T,ECTS_T, UES_T 
  where 
    ACM_T.key=ECTS_T.key 
    and ACM_T.key=UES_T.key 
    and UES_T.parcours like '%{pa}%'
    and UES_T.an = '{an}'
  group by substring(ACM_T.types,1,2) 
  """
  
spark.sql(query).show()

+------+----+-----+
|prefix|ects|hours|
+------+----+-----+
|    SO|68.0| 41.0|
|    T_| 3.0| 28.0|
|    AI| 7.0| 56.0|
|    CS| 1.5| 12.0|
|    AC|24.5|204.0|
+------+----+-----+



In [75]:

# types to UES...
# help fill competencies

query=f"""
  select ACM_T.types, 
       collect_set(concat_ws("&",UES_T.title,UES_T.key,UES_T.objective,UES_T.content))
  from ACM_T, UES_T 
  where 
    ACM_T.key=UES_T.key 
  group by ACM_T.types 
  """
  
types_to_ues=spark.sql(query)
types_to_ues.toPandas().to_csv('types-ues.csv')

#write also to my drive...
types_to_ues.toPandas().to_csv('/content/drive/MyDrive/MasterInfo2022/types-ues.csv')

types_to_ues.show(5)

+--------------------+---------------------------------------------------------+
|               types|collect_set(concat_ws(&, title, key, objective, content))|
+--------------------+---------------------------------------------------------+
|ACS_Computational...|                                     [Transport et log...|
|ACS_Specialized P...|                                     [Systèmes temps r...|
|ACS_Commputer gra...|                                     [3D Computer Grap...|
|             SOC_Pro|                                     [Bases de comptab...|
| AI_Machine learning|                                     [Machine Learning...|
+--------------------+---------------------------------------------------------+
only showing top 5 rows

